# Running a 5-Point Calibration Scan With OSO and BITE

This notebook adapts the Five Point Calibration Scan notebook to work with PSI. Running this notebook requires that a PSI namespace be spun up with `DISH_LMC_ENABLED` set to true.

## 1 Setup 

### 1.1 Environment Setup

In [ ]:
import json
import os
from time import sleep

from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions import pdm_transforms
from ska_oso_scripting.functions.sb import create_sbi, load_sbd
from ska_oso_scripting.objects import SubArray, Telescope
from tango import DevFailed, DeviceProxy

Until notebook clean up is merged need to use this for waiting for state 

In [ ]:
spinner = ["⣾", "⣽", "⣻", "⢿", "⡿", "⣟", "⣯", "⣷"]


def wait_for_state(device: DeviceProxy, desired_state, break_on_error=True) -> None:
    """Poll a tango device until either the given observation state is reached, or it throws an error.
    Arguments:
    device -- Tango Device to check
    desired_state -- The state which to break upon getting (number or state)
    break_on_error -- If set to False, will keeping running when getting an error status.
    """
    spinL = 0
    poll = 1
    while device.obsState != desired_state:
        if spinL < len(spinner) - 1:
            spinL += 1
        else:
            spinL = 0
        sleep(0.5)
        print(
            "\r",
            f"{spinner[spinL]} Poll# {poll}: Current state is {device.obsState.name}, waiting for {desired_state}...",
            end="",
        )
        if device.obsState == 9 and break_on_error:
            break
        poll += 1
    print(f"\nFinished with: {device.obsState.name}")